In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, ensemble, preprocessing, metrics, svm, cluster
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving overall_20210524.csv to overall_20210524.csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df = pd.read_csv(r'./Data/overall_20210524.csv')
df

,代號,名稱,年/月,營收成長率,營業毛利成長率,已實現銷貨毛利成長率,營業利益成長率,稅前淨利成長率,稅後淨利成長率,經常淨利成長率,常續淨利成長率,總資產成長率,淨值成長率,折舊性FA成長率,總資產報酬成長率,營收變動率,營業利益變動率,淨利變動率（單季）,稅前盈餘變動率,營業收入淨額,營業成本,營業毛利,聯屬公司已（未）實現銷貨利益,已實現銷貨毛利,營業費用,研究發展費,預期信用減損（損失）利益－營業費用,其他收益及費損淨額,營業利益,利息收入,其他收入,其他利益及損失,財務成本,採權益法之關聯企業及合資損益之份額,預期信用減損（損失）利益,除列按攤銷後成本衡量金融資產淨損益,金融資產重分類淨損益,營業外收入及支出－其他,營業外收入及支出,稅前淨利,...,負債準備－非流動,遞延貸項,應計退休金負債,遞延所得稅,非流動負債,負債總額,普通股股本,特別股股本,預收股款,待分配股票股利,換股權利證書,股本,資本公積合計,法定盈餘公積,特別盈餘公積,未分配盈餘,保留盈餘,其他權益,庫藏股票帳面值,母公司股東權益合計,非控制權益,股東權益總額,負債及股東權益總額,年份,季,單季營收(千元),單季營收成長,最高價,最低價,成交量(股),交易額(萬),開盤價,收盤價,漲跌幅(%),進場訊號,來自營運之現金流量/營業收入淨額,來自營運之現金流量/ 繼續營業單位損益,來自營運現金流量成長率,應收帳款週轉次數,存貨週轉率（次）
0,1471,首利,20-Dec,3.55,873.06,873.06,45.97,254.92,250.96,250.96,43.24,26.00,60.83,-46.1019,7.01,-43.91,-325.25,-79.91,-78.89,241949,212333,29616,0,29616,53461,2314,5040,0,-23845,185,8795,107566,2005,0,0,0,0,0,114356,90511,...,0,0,0,0,15573,551634,1504145,0,0,0,0,1504145,3539,0,0,-534665,-534665,70388,0,1043407,-40,1043367,1595001,2020,4,241949,0.377110,7.49,4.95,32241669,11508,6.00,7.30,0.233108,1,-0.109416,-0.318419,59.047383,0.80,2.41
1,1503,士電,20-Dec,10.71,-6.45,-6.45,-13.94,-22.23,-39.65,-39.65,-40.02,3.19,4.16,-0.0697,-0.36,20.88,-45.16,-46.51,-46.98,7262739,6138053,1124686,0,1124686,845686,166352,21802,0,279000,9288,10382,5390,11440,33122,0,0,0,0,37454,316454,...,19329,0,407380,2263326,3699423,16357472,5209722,0,0,0,0,5209722,2626221,2730986,5136954,8750888,16618828,717532,0,25172303,528042,25700345,42057817,2020,4,7262361,0.388073,51.40,46.70,7765527,7634,50.60,49.20,-0.031496,0,0.105486,3.316558,117.784897,1.24,1.36
2,1504,東元,20-Dec,3.44,3.16,3.16,30.24,21.68,37.02,37.02,30.38,6.64,9.59,-4.9576,0.30,8.97,21.07,15.30,4.05,12474622,9910318,2564304,325,2564629,1703907,247603,15472,0,860722,31038,135040,19954,47791,36318,0,0,0,0,143521,1004243,...,132414,0,1656948,2539962,18434871,38053839,19676929,0,0,0,0,19676929,7386901,7024635,3640779,17271503,27936917,7339258,-511710,61828295,5796937,67625232,105679071,2020,4,12385855,0.356141,33.65,26.80,291656601,146361,27.70,32.35,0.169982,1,0.190255,2.637443,4.224476,1.25,1.01
3,1506,正道,20-Dec,-13.63,184.84,184.84,59.16,74.51,73.30,73.30,62.34,-16.10,2.37,-17.2768,4.81,5.88,25.37,22.67,22.51,211845,198022,13823,0,13823,54570,7977,-958,0,-40747,103,-832,-3770,3299,3720,0,0,0,0,-4181,-44928,...,0,0,15381,100891,521112,1387043,1631618,0,0,0,0,1631618,139516,0,0,-793236,-793236,-124261,0,853637,49203,902840,2289883,2020,4,213851,0.031696,10.85,9.27,10687470,5564,10.80,9.75,-0.097222,0,0.035989,-0.169769,-85.557050,1.09,0.47
4,1507,永大,20-Dec,20.18,20.96,20.96,13.12,34.52,55.61,55.61,37.14,10.38,5.40,-2.4472,0.50,6.10,-20.61,-24.82,-36.06,4492021,3433267,1058754,0,1058754,758439,129078,26419,0,300315,14744,17090,43946,236,4578,0,0,0,0,65378,365693,...,0,0,259929,3666,546393,11874253,4108200,0,0,0,0,4108200,279398,3171035,307639,4117021,7595695,-79854,-69411,11834028,208272,12042300,23916553,2020,4,4472170,0.005277,63.60,56.10,4965266,5410,62.70,60.00,-0.043062,0,0.081834,1.079852,-10.728787,1.07,0.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18491,8249,菱光,10-Mar,121.15,871.31,871.31,167.39,263.06,245.76,245.76,242.40,16.88,4.41,0.0000,1.58,-15.53,27.17,29.80,50.37,1241468,1112589,128879,0,128879,80897,23378,0,0,47982,1100,12859,-2020,1013,0,0,0,0,0,9826,57808,...,0,0,61,779,840,1369244,1270350,0,0,0,0,1270350,677088,245855,39847,504013,789715,252526,0,2989679,0,2989679,4358923,2010,1,1241467,-0.078446,31.00,20.80,117771837,50581,28.90,28.00,-0.027778,0,0.041402,1.131912,0.000000,1.46,4.36
18492,8261,富鼎,10-Mar,39.45,21.43,21.43,56.43,-24.12,-30.99,-30.99,-5.38,5.36,7.98,0.0000,-0.88,8.26,30.54,12.94,42.95,861326,728357,132969,0,132969,68849,26099,0,0,64120,15,8288,-4668,176,0,0,0,0,0,3444,67564,...,0,0

# 新增區段

In [8]:
# Drop Unused Columns
df.drop(['最高價', '最低價', '成交量(股)', '交易額(萬)', '開盤價', '收盤價', '漲跌幅(%)', '年/月'], 1, inplace=True)

# Features Selected Manually
cols = ['營收成長率', '營業毛利成長率', '營業利益成長率', '稅前淨利成長率', '經常淨利成長率',
        '折舊性FA成長率', '研究發展費', '每股盈餘', 'ROA(C)稅前息前折舊前', 'ROE(A)－稅後',
        '營業毛利率', '營業利益率', '稅前淨利率', '來自營運之現金流量/營業收入淨額',
        '來自營運之現金流量/ 繼續營業單位損益', '來自營運現金流量成長率', '應收帳款週轉次數',
        '存貨週轉率（次）', '季', '進場訊號']

# Sort Columns
first = ['代號', '名稱', '年份']
change_cols = first + [c for c in df.columns if c not in first]
df = df[change_cols]
df.replace({np.inf: 0}, inplace=True)
df.reset_index(inplace=True, drop=True)

In [9]:
df

,代號,名稱,年份,營收成長率,營業毛利成長率,已實現銷貨毛利成長率,營業利益成長率,稅前淨利成長率,稅後淨利成長率,經常淨利成長率,常續淨利成長率,總資產成長率,淨值成長率,折舊性FA成長率,總資產報酬成長率,營收變動率,營業利益變動率,淨利變動率（單季）,稅前盈餘變動率,營業收入淨額,營業成本,營業毛利,聯屬公司已（未）實現銷貨利益,已實現銷貨毛利,營業費用,研究發展費,預期信用減損（損失）利益－營業費用,其他收益及費損淨額,營業利益,利息收入,其他收入,其他利益及損失,財務成本,採權益法之關聯企業及合資損益之份額,預期信用減損（損失）利益,除列按攤銷後成本衡量金融資產淨損益,金融資產重分類淨損益,營業外收入及支出－其他,營業外收入及支出,稅前淨利,...,避險之金融負債－非流動,按攤銷後成本衡量之金融負債－非流動,合約負債－非流動,特別股負債－非流動,應付公司債－非流動,銀行借款－非流動,其他長期借款－非流動,租賃負債－非流動,負債準備－非流動,遞延貸項,應計退休金負債,遞延所得稅,非流動負債,負債總額,普通股股本,特別股股本,預收股款,待分配股票股利,換股權利證書,股本,資本公積合計,法定盈餘公積,特別盈餘公積,未分配盈餘,保留盈餘,其他權益,庫藏股票帳面值,母公司股東權益合計,非控制權益,股東權益總額,負債及股東權益總額,季,單季營收(千元),單季營收成長,進場訊號,來自營運之現金流量/營業收入淨額,來自營運之現金流量/ 繼續營業單位損益,來自營運現金流量成長率,應收帳款週轉次數,存貨週轉率（次）
0,1471,首利,2020,3.55,873.06,873.06,45.97,254.92,250.96,250.96,43.24,26.00,60.83,-46.1019,7.01,-43.91,-325.25,-79.91,-78.89,241949,212333,29616,0,29616,53461,2314,5040,0,-23845,185,8795,107566,2005,0,0,0,0,0,114356,90511,...,0,0,0,0,0,10883,0,540,0,0,0,0,15573,551634,1504145,0,0,0,0,1504145,3539,0,0,-534665,-534665,70388,0,1043407,-40,1043367,1595001,4,241949,0.377110,1,-0.109416,-0.318419,59.047383,0.80,2.41
1,1503,士電,2020,10.71,-6.45,-6.45,-13.94,-22.23,-39.65,-39.65,-40.02,3.19,4.16,-0.0697,-0.36,20.88,-45.16,-46.51,-46.98,7262739,6138053,1124686,0,1124686,845686,166352,21802,0,279000,9288,10382,5390,11440,33122,0,0,0,0,37454,316454,...,0,0,0,0,0,850000,0,3743,19329,0,407380,2263326,3699423,16357472,5209722,0,0,0,0,5209722,2626221,2730986,5136954,8750888,16618828,717532,0,25172303,528042,25700345,42057817,4,7262361,0.388073,0,0.105486,3.316558,117.784897,1.24,1.36
2,1504,東元,2020,3.44,3.16,3.16,30.24,21.68,37.02,37.02,30.38,6.64,9.59,-4.9576,0.30,8.97,21.07,15.30,4.05,12474622,9910318,2564304,325,2564629,1703907,247603,15472,0,860722,31038,135040,19954,47791,36318,0,0,0,0,143521,1004243,...,0,0,0,0,6000000,3309400,0,4423897,132414,0,1656948,2539962,18434871,38053839,19676929,0,0,0,0,19676929,7386901,7024635,3640779,17271503,27936917,7339258,-511710,61828295,5796937,67625232,105679071,4,12385855,0.356141,1,0.190255,2.637443,4.224476,1.25,1.01
3,1506,正道,2020,-13.63,184.84,184.84,59.16,74.51,73.30,73.30,62.34,-16.10,2.37,-17.2768,4.81,5.88,25.37,22.67,22.51,211845,198022,13823,0,13823,54570,7977,-958,0,-40747,103,-832,-3770,3299,3720,0,0,0,0,-4181,-44928,...,0,0,0,0,0,384896,0,19941,0,0,15381,100891,521112,1387043,1631618,0,0,0,0,1631618,139516,0,0,-793236,-793236,-124261,0,853637,49203,902840,2289883,4,213851,0.031696,0,0.035989,-0.169769,-85.557050,1.09,0.47
4,1507,永大,2020,20.18,20.96,20.96,13.12,34.52,55.61,55.61,37.14,10.38,5.40,-2.4472,0.50,6.10,-20.61,-24.82,-36.06,4492021,3433267,1058754,0,1058754,758439,129078,26419,0,300315,14744,17090,43946,236,4578,0,0,0,0,65378,365693,...,0,0,0,0,0,0,0,8029,0,0,259929,3666,546393,11874253,4108200,0,0,0,0,4108200,279398,3171035,307639,4117021,7595695,-79854,-69411,11834028,208272,12042300,23916553,4,4472170,0.005277,0,0.081834,1.079852,-10.728787,1.07,0.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18491,8249,菱光,2010,121.15,871.31,871.31,167.39,263.06,245.76,245.76,242.40,16.88,4.41,0.0000,1.58,-15.53,27.17,29.80,50.37,1241468,1112589,128879,0,128879,80897,23378,0,0,47982,1100,12859,-2020,1013,0,0,0,0,0,9826,57808,...,0,0,0,0,0,0,0,0,0,0,61,779,840,1369244,1270350,0,0,0,0,1270350,677088,245855,39847,504013,789715,252526,0,2989679,0,2989679,4358923,1,1241467,-0.078446,0,0.041402,1.131912,0.000000,1.46,4.36
18492,8261,富鼎,2010,39.45,21.43,21.43,56.43,-24.12,-30.99,-30.99,-5.38,5.36,7.98,0.0000,-0.88,8.26,30.54,12.94,42.95,861326,728357,132969,0,132969,68849,26099,0,0,64120,15,8288,-4668,176,0,0,0,0,0,3444,67564,...,0,0,0,0,1798,0,0,0,0,0,0,0,1798,808073,1392142,0,0,0,0,1392142,547580,112001,0,260414,372415,11214,0,2323351,0,2323351,3131424,1,860516,0.079750

# Method A

In [20]:
# Data Normalization ans Split
df_a = df[first + cols]
X = preprocessing.scale(df_a.iloc[:, 3:-1].values)
y = df_a.iloc[:, -1].values

train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, test_size = 0.2)

In [22]:
# RF
forest = ensemble.RandomForestClassifier(n_estimators =120,oob_score = True,n_jobs = -1,
                                         max_depth=9,min_samples_split=120,min_samples_leaf=50,
                                         random_state=50) # 調參
forest_fit = forest.fit(train_X, train_y)

# 預測
test_y_predicted = forest.predict(test_X)

# 績效
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
roc = roc_auc_score(test_y, test_y_predicted)
print(accuracy)
print(roc)

metrics.confusion_matrix(test_y, test_y_predicted)

0.7418918918918919
0.5113837558177751


array([[2717,   17],
       [ 938,   28]])

In [23]:
for i, x in enumerate(df_a.columns[4:-1]):
    print(x, forest.feature_importances_[i])

營業毛利成長率 0.03953806275192244
營業利益成長率 0.061498882222499184
稅前淨利成長率 0.08382443190599384
經常淨利成長率 0.13442484017216585
折舊性FA成長率 0.12059213880271427
研究發展費 0.04192700063430833
每股盈餘 0.030591015828599563
ROA(C)稅前息前折舊前 0.040733197300992674
ROE(A)－稅後 0.04400717861288328
營業毛利率 0.06073077014206644
營業利益率 0.025505861120577185
稅前淨利率 0.029228114536346477
來自營運之現金流量/營業收入淨額 0.03642890307517125
來自營運之現金流量/ 繼續營業單位損益 0.03482330971547541
來自營運現金流量成長率 0.03454926766708073
應收帳款週轉次數 0.029916908435256684
存貨週轉率（次） 0.028204224670170676
季 0.02307741015865722


In [24]:
# SVM
svc = svm.SVC(kernel='rbf',random_state=1,gamma=0.1,C=1) # 調參
svc_fit = svc.fit(train_X, train_y)

# 預測
test_y_predicted = svc.predict(test_X)

# 績效
accuracy = metrics.accuracy_score(test_y, test_y_predicted)
roc = roc_auc_score(test_y, test_y_predicted)
print(accuracy)
print(roc)

metrics.confusion_matrix(test_y, test_y_predicted)

0.7394594594594595
0.5017046289270456


array([[2732,    2],
       [ 962,    4]])

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


param_grid = [ {'n_estimators':[1000],
              'max_depth':[9,50],
              'min_samples_leaf': [50,1000],
              'min_samples_split':[120,500]
         }
]
 
rfc =RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(rfc, param_grid, cv=5,
               scoring='roc_auc')
 
grid_search.fit(train_X, train_y)

grid_search.best_params_

{'max_depth': 50,
 'min_samples_leaf': 50,
 'min_samples_split': 120,
 'n_estimators': 1000}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid = {'gamma': [1,0.1,0.01,0.001], 
              }
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(train_X, train_y)
print(grid.best_estimator_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] gamma=1 .........................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......................................... gamma=1, total=  23.5s
[CV] gamma=1 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   23.5s remaining:    0.0s


[CV] .......................................... gamma=1, total=  24.2s
[CV] gamma=1 .........................................................
[CV] .......................................... gamma=1, total=  24.2s
[CV] gamma=1 .........................................................
[CV] .......................................... gamma=1, total=  23.2s
[CV] gamma=1 .........................................................
[CV] .......................................... gamma=1, total=  23.2s
[CV] gamma=0.1 .......................................................
[CV] ........................................ gamma=0.1, total=  25.5s
[CV] gamma=0.1 .......................................................
[CV] ........................................ gamma=0.1, total=  24.4s
[CV] gamma=0.1 .......................................................
[CV] ........................................ gamma=0.1, total=  24.4s
[CV] gamma=0.1 .......................................................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  7.7min finished


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [52]:
# Data Normalization ans Split
df_a = df[first + cols]
X = df_a.iloc[:, 3:-1].values
y = df_a.iloc[:, -1].values

train_X, test_X, train_y, test_y = model_selection.train_test_split(X, y, test_size = 0.2)

In [64]:
# Kmeans

for i in range(2, len(cols)):

    kmeans = cluster.KMeans(n_clusters=i)
    k = kmeans.fit_predict(X)
    df_a['Cluster'] = k

    p = 0
    for n, c in df_a['Cluster'].value_counts().items():
        if c <= 1:
            p += 1
        else:
            pass

    if p == 0:
        print(f'=====K : {i}=====')
        for x in list(set(df_a['Cluster'])):
            print(f'Cluster {x}')
            df_ac = df_a[df_a['Cluster']==x]
            df_ac.reset_index(drop=True, inplace=True)
            if len(df_ac['進場訊號'].value_counts()) < 2:
                pass
            else:
                X1 = preprocessing.scale(df_ac.iloc[:, 3:-2].values)
                y1 = df_ac.iloc[:, -2].values
                train_X, test_X, train_y, test_y = model_selection.train_test_split(X1, y1, test_size = 0.2)

                ## RF
                forest = ensemble.RandomForestClassifier(n_estimators=900, oob_score=True, n_jobs=-1,
                        max_depth=9, min_samples_split=120, min_samples_leaf=50,
                                            random_state=50)  # 調參
                forest_fit = forest.fit(train_X, train_y) 
                test_y_predicted = forest.predict(test_X)
                print('/RF/')
                print(f'Accuracy:{metrics.accuracy_score(test_y, test_y_predicted)}')
                print(f'Precision:{metrics.precision_score(test_y, test_y_predicted)}')
                print(f'Recall:{metrics.recall_score(test_y, test_y_predicted)}')
                print(f'F1-Score:{metrics.f1_score(test_y, test_y_predicted)}')
                print(f'混淆矩陣:\n{metrics.confusion_matrix(test_y, test_y_predicted)}\n')

                ## SVM
                svc = svm.SVC(kernel='rbf',random_state=1,gamma=0.1,C=1)  # 調參
                svc_fit = svc.fit(train_X, train_y)
                test_y_predicted = svc.predict(test_X)
                print('/SVM/')
                print(f'Accuracy:{metrics.accuracy_score(test_y, test_y_predicted)}')
                print(f'Precision:{metrics.precision_score(test_y, test_y_predicted)}')
                print(f'Recall:{metrics.recall_score(test_y, test_y_predicted)}')
                print(f'F1-Score:{metrics.f1_score(test_y, test_y_predicted)}')
                print(f'混淆矩陣:\n{metrics.confusion_matrix(test_y, test_y_predicted)}\n')




=====K : 2=====
Cluster 0
/RF/
Accuracy:0.7313026924122926
Precision:0.5689655172413793
Recall:0.03313253012048193
F1-Score:0.06261859582542695
混淆矩陣:
[[2656   25]
 [ 963   33]]

/SVM/
Accuracy:0.7291270057111776
Precision:0.5
Recall:0.001004016064257028
F1-Score:0.002004008016032064
混淆矩陣:
[[2680    1]
 [ 995    1]]

Cluster 1
/RF/
Accuracy:0.6521739130434783
Precision:0.0
Recall:0.0
F1-Score:0.0
混淆矩陣:
[[15  0]
 [ 8  0]]

/SVM/
Accuracy:0.6086956521739131
Precision:0.0
Recall:0.0
F1-Score:0.0
混淆矩陣:
[[14  1]
 [ 8  0]]

=====K : 3=====
Cluster 0
/RF/
Accuracy:0.7372262773722628
Precision:0.547945205479452
Recall:0.042417815482502653
F1-Score:0.07874015748031497
混淆矩陣:
[[2586   33]
 [ 903   40]]

/SVM/
Accuracy:0.7346996069623807
Precision:0.25
Recall:0.0010604453870625664
F1-Score:0.002111932418162619
混淆矩陣:
[[2616    3]
 [ 942    1]]

Cluster 1
/RF/
Accuracy:0.55
Precision:0.0
Recall:0.0
F1-Score:0.0
混淆矩陣:
[[11  0]
 [ 9  0]]

/SVM/
Accuracy:0.55
Precision:0.0
Recall:0.0
F1-Score:0.0
混淆矩陣:
